In [1]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
from qiskit import Aer
from qiskit.circuit.library import QFT


from tools.classical import *
from tools.quantum import *

In [2]:
num_qubits = 4
num_estimating_qubits = 4

X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()

# print(hamiltonian_qt)
#* HAMILTONIANS
# rescaling_factor, shift = rescaling_and_shift_factors(hamiltonian_qt)  # Shift already rescaled
# just_shift = shift * rescaling_factor                                  # Unscaled shift
# shifted_hamiltonian_qt = hamiltonian_qt + just_shift * qt.qeye(hamiltonian_qt.shape[0])
# rescaled_hamiltonian_qt = hamiltonian_qt / rescaling_factor + shift * qt.qeye(hamiltonian_qt.shape[0])

### Find Heisenberg coefficients for non degenerate spectrum

In [3]:
#TODO: Normalize spectrum 
coeff_lower_bound = 0
coeff_xx = np.arange(1, coeff_lower_bound, -0.1)
coeff_yy = np.arange(1, coeff_lower_bound, -0.1)
coeff_zz = np.arange(1, coeff_lower_bound, -0.1)

# get all combinations of coefficients
coeffs = np.array(np.meshgrid(coeff_xx, coeff_yy, coeff_zz)).T.reshape(-1, 3)

for coeff in coeffs:
    hamiltonian_qt = hamiltonian_matrix([X, X], [Y, Y], [Z, Z], coeffs=coeff, num_qubits=num_qubits, symbreak_term=[Z])
    exact_spec = np.linalg.eigvalsh(hamiltonian_qt)
    exact_spec = np.round(exact_spec, 3)

    if len(np.unique(exact_spec)) == len(exact_spec):  #FIXME: Better check how close the eigvals are, since this currently gives 1.225 and 1.236
        print("Unique spectrum found: ", exact_spec)
        print("Coefficients: ", coeff)
        break

rescaling_factor, shift = rescaling_and_shift_factors(hamiltonian_qt)
rescaled_hamiltonian_qt = hamiltonian_qt / rescaling_factor + shift * qt.qeye(hamiltonian_qt.shape[0])
rescaled_coeff = coeff / rescaling_factor
print('----')
print('Rescaled Hamiltonian spectrum: ', np.round(np.linalg.eigvalsh(rescaled_hamiltonian_qt), 3))
print('Rescaled coefficients: ', rescaled_coeff)


Applied sym breaking term onto qubit 1
Applied sym breaking term onto qubit 2
Unique spectrum found:  [-8.086 -5.236 -4.    -3.236 -1.486 -1.236 -0.764 -0.     0.764  1.225
  1.236  2.     3.236  4.348  5.236  6.   ]
Coefficients:  [1. 1. 1.]
----
Rescaled Hamiltonian spectrum:  [0.    0.202 0.29  0.344 0.469 0.486 0.52  0.574 0.628 0.661 0.662 0.716
 0.804 0.883 0.946 1.   ]
Rescaled coefficients:  [0.07099116 0.07099116 0.07099116]


In [7]:
#* Energy register check for QPE, non-degenerate and non-rescaled but chosen E = 0.764
exact_spec, eigenstates = np.linalg.eigh(rescaled_hamiltonian_qt)

eigenstate = eigenstates[8]
initial_energy = exact_spec[8]
initial_eigenstate = Statevector(eigenstate.reshape(2**num_qubits, 1))
print(f'Initial energy = {initial_energy}')

# Initialize  #TODO: Add the shift too to the quantum circuit 
qr_energy = QuantumRegister(num_estimating_qubits, name="E")
cr_energy = ClassicalRegister(num_estimating_qubits, name="crE")
qr_sys  = QuantumRegister(num_qubits, name="sys")
circ = QuantumCircuit(qr_energy, qr_sys, cr_energy)
circ.initialize(initial_eigenstate, qr_sys)
circ.h(qr_energy)

# QPE
T = 1
total_time = T * 2 * np.pi
num_trotter_steps = 100
step_size = total_time / num_trotter_steps

# Double the num_trotter_steps for T = 2 and so on...
trotter_step_circ = trotter_step_heisenberg(num_qubits, coeffs = rescaled_coeff, symbreak=True)
ham_time_evol = lambda m, t: ham_evol(num_qubits, trotter_step=trotter_step_circ, num_trotter_steps=m, time=t)
U = ham_time_evol(num_trotter_steps, total_time)
cU = U.control(1)
for q in range(num_estimating_qubits):
    for _ in range(2**q):
        circ.compose(cU, [q, *list(qr_sys)], inplace=True)

# Inverse QFT
qft_circ = QFT(num_estimating_qubits, inverse=True)  # With SWAPs
circ.compose(qft_circ, qr_energy, inplace=True)

circ.measure(qr_energy, cr_energy)
print(circ)

Initial energy = 0.7639320225002102


KeyboardInterrupt: 

In [5]:
tr_circ = transpile(circ, basis_gates=['u', 'cx'], optimization_level=3)
simulator = Aer.get_backend('statevector_simulator')
shots = 10
job = simulator.run(tr_circ, shots=shots)
counts = job.result().get_counts()
print(counts)

#! For non rescaled spectrum, under 100 Trotter steps we have garbage as we found out!


{'0000': 10}


/Users/bence/miniforge3/envs/qiskit_env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/bence/miniforge3/envs/qiskit_env/lib/python3.11/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
